In [1]:
from mpl_toolkits.basemap import Basemap  # import Basemap matplotlib toolkit
import numpy as np
import matplotlib.pyplot as plt
import pygrib # import pygrib interface to grib_api
import pandas as pd
import os
import sys
import math
import folium
import csv 
import time

start = time.time()

%matplotlib inline

root = os.getcwd()
root_data = root + "/grib_data"
root_URL = "http://database.rish.kyoto-u.ac.jp/arch/jmadata/data/gpv/original/"

#timestomp like 201808010000 means that it is at 2019/04/09/08:00(UTC timezone)
year = "2019"
month = "04"
day = "21"
roottime="120000"
FH="00-15"

def getgrib(year,month,day,roottime,FH):
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"
    file_url = root_URL\
                +str(year + "/" + month + "/" + day +"/")\
                +file_name
    root_file= "grib_data/"+file_name
    if os.path.isfile(root_file) == False:
            command = "wget -P grib_data/ "+ file_url
            os.system(command)

    getgrib_output = pygrib.open(root_file)
    return getgrib_output

def getrootgrib(year,month,day,time,FH):
    file_name = "Z__C_RJTD_"+str(year+month+day+roottime)+"_MSM_GPV_Rjp_Lsurf_FH"+str(FH)+"_grib2.bin"   
    return "grib_data/"+file_name

In [2]:
print(os.path.isfile( getrootgrib(year,month,day,roottime,FH) ))

False


In [3]:
grbs = getgrib(year,month,day,roottime,FH)
latlons = grbs

In [4]:
print(os.path.isfile( getrootgrib(year,month,day,roottime,FH) ))

True


In [5]:
#check parameters in gribfile
grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb_1.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_rad= {'lons':flat_lons, 'lats':flat_lats}
pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
pd_temp= {'lons':flat_lons, 'lats':flat_lats}

i,j,k,l =0,0,0,0

for grb in grbs:
    print(grb)
    
    if grb.name == "Downward short-wave radiation flux":
        radiation = grb.values
        flat_radiation = np.ravel(radiation)
        pd_name="radiation FH"+str(i)+"-"+str(i+1)+"W/m2(avg)"
        pd_rad[pd_name]=flat_radiation
        i=i+1

    elif grb.name == "10 metre U wind component":
        Uwind = grb.values
        flat_Uwind = np.ravel(Uwind)
        pd_name="Uwind FH"+str(j)+"m/s"
        pd_Uwind[pd_name]=flat_Uwind
        j=j+1

    elif grb.name == "10 metre V wind component":
        Vwind = grb.values
        flat_Vwind = np.ravel(Vwind)
        pd_name="Vwind FH"+str(k)+"m/s"
        pd_Vwind[pd_name]=flat_Vwind
        k=k+1

    elif grb.name == "Temperature":
        temperature = grb.values
        #華氏を摂氏に変換
        flat_temperature= np.ravel(temperature)
        flat_temperature=flat_temperature-273.115
        pd_name="temperature FH"+str(l)
        pd_temp[pd_name]=flat_temperature
        l=l+1

1:Pressure reduced to MSL:Pa (instant):regular_ll:meanSea:level 0:fcst time 0 hrs:from 201904211200
2:Surface pressure:Pa (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904211200
3:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201904211200
4:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 0 hrs:from 201904211200
5:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201904211200
6:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 0 hrs:from 201904211200
7:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904211200
8:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904211200
9:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 201904211200
10:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 0 hrs:from 2

122:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 10 hrs:from 201904211200
123:Temperature:K (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 10 hrs:from 201904211200
124:Relative humidity:% (instant):regular_ll:heightAboveGround:level 1.5 m:fcst time 10 hrs:from 201904211200
125:Low cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201904211200
126:Medium cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201904211200
127:High cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201904211200
128:Total cloud cover:% (instant):regular_ll:surface:level 0:fcst time 10 hrs:from 201904211200
129:Total precipitation:kg m-2 (accum):regular_ll:surface:level 0:fcst time 9-10 hrs (accum):from 201904211200
130:Downward short-wave radiation flux:W m**-2 (avg):regular_ll:surface:level 0:fcst time 9-10 hrs (avg):from 201904211200
131:Pressure reduced to MSL:Pa (instant):regula

In [6]:
#緯度経度の範囲を選択
east_lon=130.438
west_lon= 130.437
north_lat=33.555
south_lat=33.550

In [7]:
#range adjust
def range(df):
    df=df[df['lons'] > west_lon]
    df=df[df['lons'] < east_lon]
    df=df[df['lats'] > south_lat]
    df=df[df['lats'] < north_lat]
    return df

def range_ajust(east_lon,west_lon,north_lat,south_lat,df_all1):
    df=range(df_all1)
    return df



## Select parameter temperature, Uwind, Vwind, radiation

In [8]:
df_input1=pd.DataFrame(pd_temp)
df_input2=pd.DataFrame(pd_Uwind)
df_input3=pd.DataFrame(pd_Vwind)
df_input4=pd.DataFrame(pd_rad)
    
df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()

new_dir_path="Output_CSV/"+year+"/"+month+"/"+day
os.makedirs(new_dir_path,exist_ok=True)

temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_temp.to_csv(temp_name, encoding='utf-8', index=False)

Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_rad.to_csv(rad_name, encoding='utf-8', index=False)

df_rad

,index,lons,lats,radiation FH0-1W/m2(avg),radiation FH1-2W/m2(avg),radiation FH2-3W/m2(avg),radiation FH3-4W/m2(avg),radiation FH4-5W/m2(avg),radiation FH5-6W/m2(avg),radiation FH6-7W/m2(avg),radiation FH7-8W/m2(avg),radiation FH8-9W/m2(avg),radiation FH9-10W/m2(avg),radiation FH10-11W/m2(avg),radiation FH11-12W/m2(avg),radiation FH12-13W/m2(avg),radiation FH13-14W/m2(avg),radiation FH14-15W/m2(avg)
0,135328,130.4375,33.55,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.4375,55.643873,186.982967,373.79351,519.639143,631.791508,711.638134


In [9]:
FH="16-33"

grbs = getgrib(year,month,day,roottime,FH)
latlons = grbs

#check parameters in gribfile
grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb_1.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_rad= {'lons':flat_lons, 'lats':flat_lats}
pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
pd_temp= {'lons':flat_lons, 'lats':flat_lats}

i,j,k,l =0,0,0,0

for grb in grbs:
    #print(grb)
    
    if grb.name == "Downward short-wave radiation flux":
        radiation = grb.values
        flat_radiation = np.ravel(radiation)
        pd_name="radiation FH"+str(i+16)+"-"+str(i+17)+"W/m2(avg)"
        pd_rad[pd_name]=flat_radiation
        i=i+1

    elif grb.name == "10 metre U wind component":
        Uwind = grb.values
        flat_Uwind = np.ravel(Uwind)
        pd_name="Uwind FH"+str(j+16)+"m/s"
        pd_Uwind[pd_name]=flat_Uwind
        j=j+1

    elif grb.name == "10 metre V wind component":
        Vwind = grb.values
        flat_Vwind = np.ravel(Vwind)
        pd_name="Vwind FH"+str(k+16)+"m/s"
        pd_Vwind[pd_name]=flat_Vwind
        k=k+1

    elif grb.name == "Temperature":
        temperature = grb.values
        #華氏を摂氏に変換
        flat_temperature= np.ravel(temperature)
        flat_temperature=flat_temperature-273.115
        pd_name="temperature FH"+str(l+16)
        pd_temp[pd_name]=flat_temperature
        l=l+1
        
df_input1=pd.DataFrame(pd_temp)
df_input2=pd.DataFrame(pd_Uwind)
df_input3=pd.DataFrame(pd_Vwind)
df_input4=pd.DataFrame(pd_rad)
    
df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()

temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_temp.to_csv(temp_name, encoding='utf-8', index=False)

Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_rad.to_csv(rad_name, encoding='utf-8', index=False)

In [10]:
FH="34-39"

grbs = getgrib(year,month,day,roottime,FH)
latlons = grbs

#check parameters in gribfile
grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb_1.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_rad= {'lons':flat_lons, 'lats':flat_lats}
pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
pd_temp= {'lons':flat_lons, 'lats':flat_lats}

i,j,k,l =0,0,0,0

for grb in grbs:
    #print(grb)
    
    if grb.name == "Downward short-wave radiation flux":
        radiation = grb.values
        flat_radiation = np.ravel(radiation)
        pd_name="radiation FH"+str(i+16)+"-"+str(i+17)+"W/m2(avg)"
        pd_rad[pd_name]=flat_radiation
        i=i+1

    elif grb.name == "10 metre U wind component":
        Uwind = grb.values
        flat_Uwind = np.ravel(Uwind)
        pd_name="Uwind FH"+str(j+16)+"m/s"
        pd_Uwind[pd_name]=flat_Uwind
        j=j+1

    elif grb.name == "10 metre V wind component":
        Vwind = grb.values
        flat_Vwind = np.ravel(Vwind)
        pd_name="Vwind FH"+str(k+16)+"m/s"
        pd_Vwind[pd_name]=flat_Vwind
        k=k+1

    elif grb.name == "Temperature":
        temperature = grb.values
        #華氏を摂氏に変換
        flat_temperature= np.ravel(temperature)
        flat_temperature=flat_temperature-273.115
        pd_name="temperature FH"+str(l+16)
        pd_temp[pd_name]=flat_temperature
        l=l+1
        
df_input1=pd.DataFrame(pd_temp)
df_input2=pd.DataFrame(pd_Uwind)
df_input3=pd.DataFrame(pd_Vwind)
df_input4=pd.DataFrame(pd_rad)
    
df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()

temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_temp.to_csv(temp_name, encoding='utf-8', index=False)

Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_rad.to_csv(rad_name, encoding='utf-8', index=False)

In [11]:
FH="40-51"

grbs = getgrib(year,month,day,roottime,FH)
latlons = grbs

#check parameters in gribfile
grb_1 = grbs.select(name='Pressure reduced to MSL')[0]

# lats,lonsは二次元配列で緯度経度が入っている
lats, lons = grb_1.latlons()

#一次元に変換
# 二次元のままでは描画できない
# 2次元の格子点を1次元配列にならす
flat_lats= np.ravel(lats)
flat_lons= np.ravel(lons)

pd_rad= {'lons':flat_lons, 'lats':flat_lats}
pd_Uwind= {'lons':flat_lons, 'lats':flat_lats}
pd_Vwind= {'lons':flat_lons, 'lats':flat_lats}
pd_temp= {'lons':flat_lons, 'lats':flat_lats}

i,j,k,l =0,0,0,0

for grb in grbs:
    #print(grb)
    
    if grb.name == "Downward short-wave radiation flux":
        radiation = grb.values
        flat_radiation = np.ravel(radiation)
        pd_name="radiation FH"+str(i+16)+"-"+str(i+17)+"W/m2(avg)"
        pd_rad[pd_name]=flat_radiation
        i=i+1

    elif grb.name == "10 metre U wind component":
        Uwind = grb.values
        flat_Uwind = np.ravel(Uwind)
        pd_name="Uwind FH"+str(j+16)+"m/s"
        pd_Uwind[pd_name]=flat_Uwind
        j=j+1

    elif grb.name == "10 metre V wind component":
        Vwind = grb.values
        flat_Vwind = np.ravel(Vwind)
        pd_name="Vwind FH"+str(k+16)+"m/s"
        pd_Vwind[pd_name]=flat_Vwind
        k=k+1

    elif grb.name == "Temperature":
        temperature = grb.values
        #華氏を摂氏に変換
        flat_temperature= np.ravel(temperature)
        flat_temperature=flat_temperature-273.115
        pd_name="temperature FH"+str(l+16)
        pd_temp[pd_name]=flat_temperature
        l=l+1
        
df_input1=pd.DataFrame(pd_temp)
df_input2=pd.DataFrame(pd_Uwind)
df_input3=pd.DataFrame(pd_Vwind)
df_input4=pd.DataFrame(pd_rad)
    
df_temp = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input1).reset_index()
df_Uwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input2).reset_index()
df_Vwind = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input3).reset_index()
df_rad = range_ajust(east_lon,west_lon,north_lat,south_lat,df_input4).reset_index()

temp_name=new_dir_path+"/"+"FH_"+FH+"_"+"temperature" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_temp.to_csv(temp_name, encoding='utf-8', index=False)

Uwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Uwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Uwind.to_csv(Uwind_name, encoding='utf-8', index=False)

Vwind_name=new_dir_path+"/"+"FH_"+FH+"_"+"Vwind" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_Vwind.to_csv(Vwind_name, encoding='utf-8', index=False)

rad_name=new_dir_path+"/"+"FH_"+FH+"_"+"radiation" + year + "_" + month + "_" + day+ "_" + roottime+".csv"
df_rad.to_csv(rad_name, encoding='utf-8', index=False)

In [12]:
elapsed_time = time.time() - start

print("elapsed_time:{0}".format(elapsed_time))

elapsed_time:57.88318586349487
